# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes. Please make sure you run this notebook from an Apache Spark 2.3 notebook.

So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.2.0:

The steps are:
- pip install
- link the jars to the correct location
- restart the kernel
- start execution at the cell with the version - check

In [1]:
!pip install systemml

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191201150918-0001
KERNEL_ID = cb5afeb0-4405-48e4-a799-66198bd91a1c
  Using cached https://files.pythonhosted.org/packages/d2/ab/43e678759326f728de861edbef34b8e2ad1b1490505f20e0d1f0716c3bf4/numpy-1.17.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/54/18/d7c101d5e93b6c78dc206fcdf7bd04c1f8138a7b1a93578158fa3b132b08/scipy-1.3.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/10/5c/0e94e689de2476c4c5e644a3bd223a1c1b9e2bdb7c510191750be74fa786/Pillow-6.2.1-cp36-cp36m-manylinux1_x86_64.w

Now we need to create two sym links that the newest version is picket up - this is a workaround and will be removed soon

In [2]:
!ln -s -f ~/user-libs/python3.6/systemml/systemml-java/systemml-1.2.0-extra.jar ~/user-libs/spark2/systemml-1.2.0-extra.jar
!ln -s -f ~/user-libs/python3.6/systemml/systemml-java/systemml-1.2.0.jar ~/user-libs/spark2/systemml-1.2.0.jar

Now please restart the kernel and make sure the version is correct

In [3]:
from systemml import MLContext
ml = MLContext(spark)
print(ml.version())
    
if not ml.version() == '1.2.0':
    raise ValueError('please upgrade to SystemML 1.2.0, or restart your Kernel (Kernel->Restart & Clear Output)')

1.2.0


Congratulations, if you see version 1.2.0, please continue with the notebook...

In [4]:
from systemml import MLContext, dml
import numpy as np
import time

Then we create an MLContext to interface with Apache SystemML. Note that we pass a SparkSession object as parameter so SystemML now knows how to talk to the Apache Spark cluster

In [5]:
ml = MLContext(spark)

Now we create some large random matrices to have numpy and SystemML crunch on it

In [6]:
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are unfamiliar with matrxi-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [7]:
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print (time.time()-start)

1.871474027633667


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [8]:
#u = np.random.rand(10000,100000)
#s = np.random.rand(100000,10000)
#w = np.random.rand(10000,10000)

# quiz week 2

In [13]:
from systemml import dml, MLContext
import numpy as np


In [14]:
ml = MLContext(spark)

In [15]:
script="""
c = sum(a %*% t(b))
"""

In [17]:
a=np.array([[1,2,3]])
b=np.array([[4,5,6]])

prog = dml(script).input('a',a).input('b',b).output('c')
c = ml.execute(prog).get('c')

SystemML Statistics:
Total execution time:		0.012 sec.
Number of executed Spark inst:	2.




In [19]:
c

32.0

In [18]:
np.sum(a.dot(b.T))

32